# Estimativa de contagens

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

No capítulo anterior, resolvemos problemas que envolvem a estimativa de proporções.

No problema do Euro, estimamos a probabilidade de uma moeda cair, e nos exercícios, estimamos uma média de batimento, a fracção de pessoas que fazem batota nos seus impostos, e a hipótese de abater um estrangeiro invasor.

Claramente, alguns destes problemas são mais realistas do que outros, e alguns são mais úteis do que outros.

Neste capítulo, trabalharemos sobre problemas relacionados com a contagem, ou a estimativa do tamanho de uma população.

Mais uma vez, alguns dos exemplos vão parecer tolos, mas alguns deles, como o problema do Tanque Alemão, têm aplicações reais, por vezes em situações de vida ou morte.

## O problema do comboio

Encontrei o problema do comboio 

em Frederick Mosteller's, [*Fififty Challenging Problems in

  Probabilidade com Soluções*](https://store.doverpublications.com/0486653552.html):

> "Uma via férrea numera as suas locomotivas por ordem 1...N. Um dia vê uma locomotiva com o número 60.  Faça uma estimativa do número de locomotivas que o caminho-de-ferro tem".

Com base nesta observação, sabemos que a ferrovia tem 60 ou mais

locomotivas.  Mas quantas mais?  Para aplicar o raciocínio Bayesiano, nós

pode quebrar este problema em duas etapas:

* O que sabíamos sobre $N$ antes de vermos os dados?

* Para qualquer valor dado de $N$, qual é a probabilidade de ver os dados (uma locomotiva com o número 60)?

A resposta à primeira pergunta é a anterior.  A resposta à

segundo é a probabilidade.

Não temos muita base para escolher um anterior, por isso vamos começar com

algo simples e depois considerar alternativas.

Vamos assumir que $N$ é igualmente provável que seja qualquer valor de 1 a 1000.

Aqui está a distribuição prévia:

In [4]:
import numpy as np
from empiricaldist import Pmf

hypos = np.arange(1, 1001)
prior = Pmf(1, hypos)

Agora vamos descobrir a probabilidade dos dados.

Numa frota hipotética de locomotivas $N$, qual é a probabilidade de vermos o número 60?

Se assumirmos que é igualmente provável que vejamos qualquer locomotiva, a hipótese de ver qualquer uma em particular é de $1/N$.

Aqui está a função que faz a actualização:

In [5]:
def update_train(pmf, data):
    """Update pmf based on new data."""
    hypos = pmf.qs
    likelihood = 1 / hypos
    impossible = (data > hypos)
    likelihood[impossible] = 0
    pmf *= likelihood
    pmf.normalize()

Esta função pode parecer familiar; é a mesma que a função de actualização para o problema dos dados no capítulo anterior.

Em termos de probabilidade, o problema do comboio é o mesmo que o problema dos dados.

Aqui está a actualização:

In [6]:
data = 60
posterior = prior.copy()
update_train(posterior, data)

Eis como se parece o posterior:

In [7]:
from utils import decorate

posterior.plot(label='Posterior after train 60', color='C4')
decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Posterior distribution')

Não surpreende que todos os valores de $N$ abaixo de 60 tenham sido eliminados.

O valor mais provável, se tivesse de adivinhar, é 60.

In [8]:
posterior.max_prob()

Isso pode não parecer um bom palpite; afinal de contas, quais são as hipóteses de ter acabado de ver o comboio com o número mais elevado?

No entanto, se quiser maximizar a hipótese de obter

a resposta exactamente certa, deve adivinhar 60.

Mas talvez esse não seja o objectivo certo.

Uma alternativa é calcular a média da distribuição posterior.

Dado um conjunto de quantidades possíveis, $q_i$, e as suas probabilidades, $p_i$, a média da distribuição é:

$$$mathrm{mean} = ^sum_i p_i q_i q_i$$

Que podemos calcular desta forma:

In [9]:
np.sum(posterior.ps * posterior.qs)

Ou podemos utilizar o método fornecido por `Pmf`:

In [10]:
posterior.mean()

A média do posterior é 333, pelo que poderá ser um bom palpite se quiser minimizar o erro.

Se jogasse este jogo de adivinhação vezes sem conta, utilizando a média do posterior como a sua estimativa minimizaria o [mean squared error](http://en.wikipedia.org/wiki/Minimum_mean_square_error) a longo prazo.

## Sensibilidade ao Prior

O anterior I utilizado na secção anterior é uniforme de 1 a 1000, mas não ofereci qualquer justificação para escolher uma distribuição uniforme ou aquele limite superior em particular.

Podemos perguntar-nos se a distribuição posterior é sensível à anterior.

Com tão poucos dados - apenas uma observação - é.

Esta tabela mostra o que acontece à medida que variamos o limite superior:

In [11]:
import pandas as pd

df = pd.DataFrame(columns=['Posterior mean'])
df.index.name = 'Upper bound'

for high in [500, 1000, 2000]:
    hypos = np.arange(1, high+1)
    pmf = Pmf(1, hypos)
    update_train(pmf, data=60)
    df.loc[high] = pmf.mean()
    
df

Como variamos o limite superior, a média posterior muda substancialmente.

Portanto, isso é mau.  

Quando o posterior é sensível ao anterior, há duas maneiras de proceder:

* Obter mais dados.

* Obter mais informação de fundo e escolher uma melhor antecedência.

Com mais dados, as distribuições posteriores baseadas em diferentes antecedentes tendem a convergir.  

Por exemplo, suponha que para além do comboio 60 vemos também os comboios 30 e 90.

Eis como os meios posteriores dependem do limite superior do anterior, quando observamos três comboios:

In [12]:
df = pd.DataFrame(columns=['Posterior mean'])
df.index.name = 'Upper bound'

dataset = [30, 60, 90]

for high in [500, 1000, 2000]:
    hypos = np.arange(1, high+1)
    pmf = Pmf(1, hypos)
    for data in dataset:
        update_train(pmf, data)
    df.loc[high] = pmf.mean()
    
df

As diferenças são menores, mas aparentemente três comboios não são suficientes para que os posteriors converjam.

## Prior da Lei do Poder

Se não houver mais dados disponíveis, outra opção é melhorar a

priores, recolhendo mais informação de base.

Provavelmente não é razoável supor que uma empresa ferroviária com 1000 locomotivas seja tão provável como uma empresa com apenas 1.

Com algum esforço, poderíamos provavelmente encontrar uma lista de empresas que

operar locomotivas na área de observação.

Ou podemos entrevistar um especialista em transporte ferroviário para recolher informações sobre a dimensão típica das empresas.

Mas mesmo sem entrarmos nas especificidades da economia ferroviária, nós

pode fazer algumas suposições instruídas.

Na maioria dos campos, existem muitas pequenas empresas, menos empresas médias, e apenas uma ou duas empresas muito grandes.

De facto, a distribuição das dimensões das empresas tende a seguir uma lei de poder, como Robert Axtell relata em *Science* (<http://www.sciencemag.org/content/293/5536/1818.full.pdf>).

Esta lei sugere que se houver 1000 empresas com menos de

10 locomotivas, pode haver 100 empresas com 100 locomotivas,

10 empresas com 1000, e possivelmente uma empresa com 10.000 locomotivas.

Matematicamente, uma lei de poder significa que o número de empresas com um determinado tamanho, $N$, é proporcional a $(1/N)^{\alpha}$, onde $\alpha$ é um parâmetro que se aproxima frequentemente de 1.

Podemos construir uma lei de poder anterior como esta:

In [13]:
alpha = 1.0
ps = hypos**(-alpha)
power = Pmf(ps, hypos, name='power law')
power.normalize()

Para comparação, aqui está novamente o anterior uniforme.

In [14]:
hypos = np.arange(1, 1001)
uniform = Pmf(1, hypos, name='uniform')
uniform.normalize()

Eis como se parece um prior da lei do poder, em comparação com o anterior uniforme:

In [15]:
uniform.plot(color='C4')
power.plot(color='C1')

decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Prior distributions')

Aqui está a actualização para ambos os antecedentes.

In [16]:
dataset = [60]
update_train(uniform, dataset)
update_train(power, dataset)

E aqui estão as distribuições posteriores.

In [17]:
uniform.plot(color='C4')
power.plot(color='C1')

decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Posterior distributions')

A lei do poder dá menos probabilidade prévia a valores elevados, o que produz meios posteriores mais baixos, e menos sensibilidade ao limite superior.

Eis como os meios posteriores dependem do limite superior quando usamos uma lei de potência anterior e observamos três comboios:

In [18]:
df = pd.DataFrame(columns=['Posterior mean'])
df.index.name = 'Upper bound'

alpha = 1.0
dataset = [30, 60, 90]

for high in [500, 1000, 2000]:
    hypos = np.arange(1, high+1)
    ps = hypos**(-alpha)
    power = Pmf(ps, hypos)
    for data in dataset:
        update_train(power, data)
    df.loc[high] = power.mean()
    
df

Agora as diferenças são muito menores.  De facto, as diferenças são muito menores,

com um limite superior arbitrariamente grande, a média converge para 134.

Assim, a lei do poder anterior é mais realista, porque se baseia em

informação geral sobre a dimensão das empresas, e comporta-se melhor na prática.

## Intervalos Credíveis

Até agora, vimos duas formas de resumir uma distribuição posterior: o valor com a maior probabilidade posterior (o MAP) e a média posterior.

Estas são ambas ** estimativas de pontos**, ou seja, valores únicos que estimam a quantidade em que estamos interessados.

Outra forma de resumir uma distribuição posterior é com percentis.

Se tiver feito um teste padronizado, poderá estar familiarizado com percentis.

Por exemplo, se a sua pontuação for o percentil 90, isso significa que fez tão bem ou melhor do que 90% das pessoas que fizeram o teste.

Se nos for dado um valor, `x`, podemos calcular o seu **percentile rank** encontrando todos os valores inferiores ou iguais a `x` e somando as suas probabilidades.

O `Pmf` fornece um método que faz este cálculo.

Assim, por exemplo, podemos calcular a probabilidade de a empresa ter menos ou igual a 100 comboios:

In [19]:
power.prob_le(100)

Com uma lei de poder anterior e um conjunto de dados de três comboios, o resultado é de cerca de 29%.

Assim, 100 comboios é o 29º percentil.

Seguindo o outro caminho, suponha que queremos calcular um determinado percentil; por exemplo, a mediana de uma distribuição é o 50º percentil.

Podemos calculá-lo somando as probabilidades até o total exceder 0,5.

Aqui está uma função que o faz:

In [20]:
def quantile(pmf, prob):
    """Compute a quantile with the given prob."""
    total = 0
    for q, p in pmf.items():
        total += p
        if total >= prob:
            return q
    return np.nan

O laço utiliza `itens', que itera as quantidades e probabilidades na distribuição.

Dentro do laço somamos as probabilidades das quantidades em ordem.

Quando o total for igual ou superior a `prob', devolvemos a quantidade correspondente.

Esta função é chamada `quantile` porque calcula um quantil em vez de um percentil.

A diferença é a forma como especificamos `prob`.

Se `prob` for uma percentagem entre 0 e 100, chamamos à quantidade correspondente um percentil.

Se `prob` for uma probabilidade entre 0 e 1, chamamos à quantidade correspondente um **quantile***.

Eis como podemos utilizar esta função para calcular o percentil 50 da distribuição posterior:

In [21]:
quantile(power, 0.5)

O resultado, 113 comboios, é a mediana da distribuição posterior.

O `Pmf' fornece um método chamado `quantile' que faz a mesma coisa.

Podemos chamar-lhe assim para calcular os percentis 5 e 95:

In [22]:
power.quantile([0.05, 0.95])

O resultado é o intervalo de 91 a 243 comboios, o que implica:

* A probabilidade é de 5% de o número de comboios ser inferior ou igual a 91.

* A probabilidade é de 5% de o número de comboios ser superior a 243.

Por conseguinte, a probabilidade é de 90% de o número de comboios se situar entre 91 e 243 (excluindo 91 e incluindo 243).

Por esta razão, a este intervalo chama-se um intervalo de 90%** credível**.

O `Pmf' também fornece o `intervalo_crível', que calcula um intervalo que contém a probabilidade dada.

In [23]:
power.credible_interval(0.9)

## O Problema do Tanque Alemão

Durante a Segunda Guerra Mundial, a Divisão de Guerra Económica dos Estados Unidos da América

A embaixada em Londres utilizou a análise estatística para estimar o alemão

produção de tanques e outros equipamentos.

Os Aliados ocidentais tinham capturado livros de registo, inventários e reparações

registos que incluíam os números de série do chassis e do motor para cada

tanques.

A análise destes registos indicou que foram atribuídos números de série

por fabricante e tipo de tanque em blocos de 100 números, esses números

em cada bloco foram utilizados sequencialmente, e que nem todos os números em cada

foram utilizados blocos.  Assim, o problema de estimar a produção alemã de tanques

poderia ser reduzido, dentro de cada bloco de 100 números, a uma forma de

problema do comboio.

Com base nesta visão, os analistas americanos e britânicos produziram

estimativas substancialmente inferiores às estimativas de outras formas

da inteligência.  E, após a guerra, os registos indicavam que eles eram

substancialmente mais precisos.

Realizaram análises semelhantes para pneus, camiões, foguetes, e outros

equipamento, produzindo uma inteligência económica precisa e accionável.

O problema do tanque alemão é historicamente interessante; é também um bom

exemplo de aplicação no mundo real da estimativa estatística.

Para mais informações sobre este problema, ver [this Wikipedia page](https://en.wikipedia.org/wiki/German_tank_problem) e Ruggles and Brodie, "An Empirical Approach to Economic Intelligence in World War II", *Journal of the American Statistical Association*, Março de 1947, [available here](https://web.archive.org/web/20170123132042/https://www.cia.gov/library/readingroom/docs/CIA-RDP79R01001A001300010013-3.pdf).

## Priores Informativos

Entre os Bayesianos, há duas abordagens para a escolha prévia

distribuições.  Alguns recomendam a escolha do anterior que melhor represente

informação de base sobre o problema; nesse caso, o anterior

é dito ser **informativo***.  O problema da utilização de um informativo

anterior é que as pessoas possam ter informações diferentes ou

interpretá-lo de forma diferente.  Por isso, os antecedentes informativos podem parecer arbitrários.

A alternativa é o chamado **uninformativo prévio**, que é

destinados a serem tão irrestritos quanto possível, a fim de permitir que os dados

falar por si próprio.  Em alguns casos, é possível identificar um anterior único

que tem alguns bens desejáveis, como representar o mínimo prévio

informação sobre a quantidade estimada.

Os priores não-informativos são apelativos porque parecem mais

objectivo.  Mas sou geralmente a favor da utilização de antecedentes informativos.

Porquê?  Primeiro, a análise Bayesiana é sempre baseada em

decisões de modelagem.  A escolha do anterior é uma dessas decisões, mas

não é o único, e pode até não ser o mais subjectivo.

Assim, mesmo que um anterior não informativo seja mais objectivo, toda a análise continua a ser subjectiva.

Além disso, para a maioria dos problemas práticos, é provável que esteja em um de dois

situações: ou tem muitos dados ou não tem muito.  Se tiver muitos dados, a escolha do anterior não importa;

priores informativos e não-informativos produzem quase os mesmos resultados.

Se não tiver muitos dados, a utilização de informação de base relevante (como a distribuição da lei do poder) faz uma grande diferença.

E se, como no problema do tanque alemão, tiver de fazer vida ou morte

decisões com base nos seus resultados, deverá provavelmente utilizar todos os

informação à sua disposição, em vez de manter a ilusão de

objectividade, fingindo saber menos do que você.

## Resumo

Este capítulo introduz o problema do comboio, que acaba por ter a mesma função de probabilidade do problema dos dados, e que pode ser aplicado ao problema do Tanque Alemão.

Em todos estes exemplos, o objectivo é estimar uma contagem, ou o tamanho de uma população.

No próximo capítulo, introduzirei "probabilidades" como uma alternativa às probabilidades, e a Regra de Bayes como uma forma alternativa ao Teorema de Bayes.

Vamos calcular a distribuição de somas e produtos, e utilizá-los para estimar o número de membros do Congresso que são corruptos, entre outros problemas.

Mas primeiro, talvez queira trabalhar nestes exercícios.

## Exercícios

**Exercício:** Suponha que está a dar uma palestra numa grande sala de conferências e o bombeiro interrompe porque pensam que a audiência ultrapassa 1200 pessoas, que é a capacidade segura da sala. 

Pensa que há menos de 1200 pessoas, e oferece-se para o provar.

Demoraria demasiado tempo a contar, por isso tenta-se uma experiência:

* Pergunta-se quantas pessoas nasceram a 11 de Maio e duas pessoas levantam as mãos.  

* Pergunta-se quantos nasceram a 23 de Maio e 1 pessoa levanta a mão.  

* Finalmente, pergunta-se quantos nasceram a 1 de Agosto, e ninguém levanta a mão.

Quantas pessoas há na audiência?  Qual é a probabilidade de haver mais de 1200 pessoas.

Dica: Lembre-se da distribuição binomial.

In [24]:
# Solution goes here

In [25]:
# Solution goes here

In [26]:
# Solution goes here

In [27]:
# Solution goes here

In [28]:
# Solution goes here

In [29]:
# Solution goes here

**Exercício:** Vejo frequentemente [rabbits](https://en.wikipedia.org/wiki/Eastern_cottontail) no jardim atrás da minha casa, mas não é fácil distingui-los, por isso não sei realmente quantos são.

Suponhamos que eu implanto um sensor de movimento [camera trap](https://en.wikipedia.org/wiki/Camera_trap) que tira uma fotografia do primeiro coelho que ele vê todos os dias.  Após três dias, comparo as fotografias e concluo que dois deles são o mesmo coelho e que o outro é diferente.

Quantos coelhos visitam o meu jardim?

Para responder a esta pergunta, temos de pensar sobre a distribuição prévia e a probabilidade dos dados:

* Já vi por vezes quatro coelhos ao mesmo tempo, por isso sei que são pelo menos assim tantos.  Ficaria surpreendido se houvesse mais de 10.  Portanto, pelo menos como ponto de partida, penso que um uniforme anterior de 4 a 10 é razoável.

* Para manter as coisas simples, vamos assumir que todos os coelhos que visitam o meu jardim são igualmente susceptíveis de serem apanhados pela armadilha da câmara num determinado dia.  Vamos também assumir que é garantido que a armadilha fotográfica recebe uma imagem todos os dias.

In [30]:
# Solution goes here

In [31]:
# Solution goes here

In [32]:
# Solution goes here

**Exercício:** Suponha que no sistema de justiça criminal, todas as penas de prisão ou são de 1, 2, ou 3 anos, com um número igual de cada uma.  Um dia, visita-se uma prisão e escolhe-se um prisioneiro ao acaso.  Qual é a probabilidade de cumprir uma pena de 3 anos?  Qual é a média da pena restante dos prisioneiros que observa?

In [33]:
# Solution goes here

In [34]:
# Solution goes here

In [35]:
# Solution goes here

**Exercício:** Se escolhi um adulto aleatório nos EUA, qual é a probabilidade de eles terem um irmão?  Para ser mais preciso, qual é a probabilidade de a mãe deles ter tido pelo menos um outro filho.

[This article from the Pew Research Center](https://www.pewsocialtrends.org/2015/05/07/family-size-among-mothers/) fornece alguns dados relevantes.  

A partir dele, extraí a seguinte distribuição de tamanho familiar para mães nos EUA que tinham 40-44 anos de idade em 2014:

In [36]:
import matplotlib.pyplot as plt

qs = [1, 2, 3, 4]
ps = [22, 41, 24, 14]
prior = Pmf(ps, qs)
prior.bar(alpha=0.7)

plt.xticks(qs, ['1 child', '2 children', '3 children', '4+ children'])
decorate(ylabel='PMF',
         title='Distribution of family size')

Para simplificar, vamos assumir que todas as famílias da categoria 4+ têm exactamente 4 filhos.

In [37]:
# Solution goes here

In [38]:
# Solution goes here

In [39]:
# Solution goes here

**Exercício:** O [Doomsday argument](https://en.wikipedia.org/wiki/Doomsday_argument) é "um argumento probabilístico que afirma prever o número de futuros membros da espécie humana dada uma estimativa do número total de seres humanos nascidos até agora".

Suponhamos que existem apenas dois tipos de civilizações inteligentes que podem acontecer no universo.  O tipo "efémero" exinua após o nascimento de apenas 200 mil milhões de indivíduos.  Os do tipo "de vida longa" sobrevivem até ao nascimento de 2.000 biliões de indivíduos.

E suponha que os dois tipos de civilização são igualmente prováveis.

Em que tipo de civilização pensa que vivemos?  

O argumento do Doomsday diz que podemos usar o número total de humanos nascidos até agora como dados.

De acordo com o [Population Reference Bureau](https://www.prb.org/howmanypeoplehaveeverlivedonearth/), o número total de pessoas que já viveram é de cerca de 108 mil milhões.

Uma vez que nasceu muito recentemente, vamos supor que é, de facto, o ser humano número 108 mil milhões.

Se $N$ for o número total que alguma vez viverá e o considerarmos uma pessoa escolhida aleatoriamente, é igualmente provável que possa ter sido a pessoa 1, ou $N$, ou qualquer número intermédio.

Então qual é a probabilidade de ser o número 108 mil milhões?

Tendo em conta estes dados e o anterior duvidoso, qual é a probabilidade de a nossa civilização ser de curta duração?

In [40]:
# Solution goes here

In [41]:
# Solution goes here

In [42]:
# Solution goes here